In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
df_o = pd.read_csv('U.S._Life_Expectancy_at_Birth_by_State_and_Census_Tract_-_2010-2015.csv', encoding='latin-1')

In [3]:
df_o.head(5)

,State,County,CensusTract,LifeExpectancy,LifeExpectancyRange,LifeExpectancyStandardError
0,Alabama,(blank),NaN,75.5,75.2-77.5,0.0328
1,Alabama,"Autauga County, AL",201.0,73.1,56.9-75.1,2.2348
2,Alabama,"Autauga County, AL",202.0,76.9,75.2-77.5,3.3453
3,Alabama,"Autauga County, AL",203.0,NaN,NaN,NaN
4,Alabama,"Autauga County, AL",204.0,75.4,75.2-77.5,1.0216


In [4]:
#get rid of rows like Alabama	(blank)	NaN	75.5	75.2-77.5	0.0328
df2 = df_o[~df_o.eq('(blank)').any(axis=1)].copy()
#split LifeExpectancyRange column into low and high float columns and drop it
df2[['LifeExpectancyLow', 'LifeExpectancyHigh']] = (df2['LifeExpectancyRange'].str.split('-',expand=True).astype(float))
df = df2.drop(columns=["LifeExpectancyRange"]).copy()

In [5]:
print(df.isna().sum())

State                             0
County                            0
CensusTract                       0
LifeExpectancy                 5922
LifeExpectancyStandardError    5922
LifeExpectancyLow              5922
LifeExpectancyHigh             5922
dtype: int64


In [8]:
df['County'] = df['County'].str.split(',').str[0].str.strip()

In [9]:
df.head(5)

,State,County,CensusTract,LifeExpectancy,LifeExpectancyStandardError,LifeExpectancyLow,LifeExpectancyHigh
1,Alabama,Autauga County,201.0,73.1,2.2348,56.9,75.1
2,Alabama,Autauga County,202.0,76.9,3.3453,75.2,77.5
3,Alabama,Autauga County,203.0,NaN,NaN,NaN,NaN
4,Alabama,Autauga County,204.0,75.4,1.0216,75.2,77.5
5,Alabama,Autauga County,205.0,79.4,1.1768,77.6,79.5


In [10]:
#df["LifeExpectancyLow"] = df["LifeExpectancyLow"].fillna(df["LifeExpectancyLow"].mean())
#df["LifeExpectancyHigh"] = df["LifeExpectancyHigh"].fillna(df["LifeExpectancyHigh"].mean())
#df["LifeExpectancy"] = df["LifeExpectancy"].fillna(df["LifeExpectancy"].mean())
#df["LifeExpectancyStandardError"] = df["LifeExpectancyStandardError"].fillna(df["LifeExpectancyStandardError"].mean())

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73070 entries, 1 to 73120
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   State                        73070 non-null  object 
 1   County                       73070 non-null  object 
 2   CensusTract                  73070 non-null  float64
 3   LifeExpectancy               67148 non-null  float64
 4   LifeExpectancyStandardError  67148 non-null  float64
 5   LifeExpectancyLow            67148 non-null  float64
 6   LifeExpectancyHigh           67148 non-null  float64
dtypes: float64(5), object(2)
memory usage: 4.5+ MB


In [12]:
db_file = "life_expectancy_geo_full.db"
table = "life_expectancy_geography_full"
conn = sqlite3.connect(db_file)
df.to_sql(table, conn, if_exists="replace", index=False)
conn.close()